In [1]:
import numpy as np
import math
import scipy as sp
from scipy.integrate import quad, dblquad

So I've tried to implement a mesh refinement algorithm in another notebook I have that is primarily aimed at solving the PDE, but it's bugged and I think at this point it's a problem of organisation. So I'll try to solve this problem separately here so that I can focus on debbuging the actual PDE solver in the other notebook.

Ultimately what I'd like to do is to only input the nodes, and the mesh generates itself. Or better even, I just give the dimensions of a square, wether I want it with a corner or not and if so, the angle of the corner. 
As much as possible I will try to re use functions so that I dont have to write too many of them. So far I needed 5 functions to construct a square mesh of any given side length.

Mesh Generating Functions for a Square Mesh:

In [2]:
def add_index (liste):
    #turns an array : [ [n_1,a_1,b_1,c_1] , [n_2,a_2,b_2,c_2] , ..., [n_k,a_k,b_k,c_k]]
    #into an indexed array : [ [1,a_1,b_1,c_1] , [2,a_2,b_2,c_2] , ..., [k,a_k,b_k,c_k] ]
    l=len(liste)
    new=[]
    for i in range(l):
        temp=liste[i]
        temp[0]=i+1
        new.append(temp)
    return new

In [3]:
def generate_coordinates(array):
    #takes in an array of x values and generates a grid 
    #returns an indexed list of coordinates
    grid=[] #initializing
    for j in range (len(array)): #now we generate the square mesh
        for i in range(len(array)):
            grid.append([i, array[i], array[j]])
    grid=add_index(grid)
    return grid

In [4]:
def generate_rectangles (l, grid):
    #takes in an array of nodes in a square and constructs subrectangles
    #returns an indexed list of nodes 
    rectangles=[]
    L=l-1
    for i in range(L): #we know we'll get L*L squares
        for j in range(L):
            rectangles.append([i,grid[j+i*l][0],grid[j+1+i*l][0],grid[j+1+(i+1)*l][0],grid[j+(i+1)*l][0]])
            #the indexing is compliacted because we have to make sure the 4 point we put together
            #are actually next to each other
    rectangles=add_index(rectangles)
    return rectangles

In [5]:
def generate_edges(shapes):
    #takes in an indexed list of shapes, I'm trying to have this work for triangles as well
    #returns a list of edges (without copies)
    edges=[]
    for i in range(len(shapes)):
        p=len(shapes[i])-1 
        for j in range(p):
            append=1
            temp=[shapes[i][1+j%p],shapes[i][1+(j+1)%p]]
            rev_temp=[shapes[i][1+(j+1)%p],shapes[i][1+j%p] ]
            if len(edges)==0: #the for-loops don't work if l=0 unfortunately
                append=1
            else:
                for k in range(len(edges)):
                    if edges[k] == temp: #check if we already have that edge
                        append=0
                    if edges[k]==rev_temp: #or if we have the inverted direction edge
                        append=0
            if append ==1:
                edges.append(temp) #if we dont, add it
    return edges

In [6]:
def generate_boundary (edges, grid):
    #takes in the list of edge in the mesh and the array of coordinates of the nodes
    #returns an array [neummann, dirichlet]
    neumann=[]
    dirichlet=[]
    for i in range(len(edges)):
        a=grid[edges[i][0]-1]#start point
        b=grid[edges[i][1]-1]#end point of the edge
        if a[1]==0.0:
            if b[1]==0.0: #check if edge lies along y-axis
                dirichlet.append(edges[i])
        if a[1]==dimension:
            if b[1]==dimension: #check if edge lies along x=dimension
                neumann.append(edges[i])
        if a[2]==dimension:
            if b[2]==dimension: #check if edge lies along y=dimension
                dirichlet.append(edges[i])
        if a[2]==0.0:
            if b[2]==0.0: #check if edge lies along x-axis
                neumann.append(edges[i])
    return ([neumann, dirichlet])

In [7]:
def square_mesh (dimension):
    #generates a square mesh of side length = dimension
    #returns a list of arrays
    array=np.linspace(0, dimension, math.ceil(dimension)+1) #we first generate the coordinates in 1D
    grid= generate_coordinates(array) #then we generate the grid
    triangles=[] #there are no triangles in a square mesh
    rectangles=generate_rectangles (len(array), grid) #we use it to construct the subrectangles
    edges=generate_edges(rectangles)#collect the edges
    boundary=generate_boundary (edges, grid)#construct the boundaries
    #boundary[0] : neumann   ;   boundary[1] : dirichlet  
    ##
    return [grid, triangles, rectangles, edges, boundary[0], boundary[1]]

Now we will try to generate a mesh with a re-entrant corner such that the user can input any angle (between 0 and 180 degrees). If I restrict the corner to be at the bottom right of the mesh, think what I can do is split it into 2 meshes : the bottom right 3 cubes and the rest of the mesh. The bottom right will then be four triangles and the rest of the mesh will be a collection of squares. Actually maybe I could do everything like a square mesh and then delete the data for the bottom right corner. Note the particular case when angle ==90, I'll just remove a square at the bottom.

Functions to Turn a Square Mesh into a Mesh with a Corner :

In [8]:
def find_and_remove_edges (l,liste):
    #very specific function but I use it twice
    #takes in a list of edges and the collection of triangles
    # removes from that list the 2 bottom right edges
    test_list=[[l,l-1],[l,2*l]]
    index_list=[]
    for i in range(2):#then we find the egdes we have to remove
        test=test_list[i]
        for j in range(len(liste)):
            temp=liste[j]
            if test[0]==temp[0]:
                if test[1]==temp[1]:
                    index_list.append(j)
            if test[1]==temp[0]:
                if test[0]==temp[1]:
                    index_list.append(j)
    index_list.sort() #arranges the list increasing order
    for i in range(len(index_list)): #then we remove them
        I=len(index_list)-1-i#we start from the end
        liste=np.delete(liste, index_list[I], 0)
    return liste

In [9]:
def pushback_index(liste):
    #takes in an indexed list of nodes, and the index of the removed_node
    #returns same list but every node k>4 becomes k-1
    node=math.ceil(dimension)+1
    new=[]
    for i in range(len(liste)):
        temp=liste[i]
        if len(temp)==2: #if that length is 2 then it's an edgel list so no index
            copy=[]
            for j in range(len(temp)):
                test=temp[j]
                if test>node:
                    test=test-1
                copy.append(test)
        else: #else its either rectangles or triangles and we have to take care of the index
            copy=[i+1]
            for j in range(len(temp)-1):
                test=temp[1+j]
                if test>node:
                    test=test-1
                copy.append(test)
        new.append(copy)
    return new

In [10]:
def adjust_grid(l, angle, grid):
    #takes in a square mesh and an angle in degrees
    #changes the 3 bottom right node's coordinates so that the mesh has a 
    #re-entrant corner of degree = angle
    phi=45-angle*1/2
    step=grid[l-2][1]-grid[l-3][1]
    x=abs(math.tan(phi)*step)
    grid[l-2][1]=grid[l-2][1]+phi/abs(phi)*x
    grid[2*l-1][2]=grid[2*l-1][2]-phi/abs(phi)*x
    grid=np.delete(grid, l-1,0)#then we delete the bottom right node
    grid=add_index (grid.tolist())
    return grid

In [11]:
def generate_triangles (rectangles, l):
    #takes in a collectino of squares in a square mesh
    #returns a list of four rectangles in the bottom right corner
    triangles=[]
    change_list=[rectangles[l-3], rectangles[2*(l-1)-1]]
    for i in range(2):
        diagonal=[change_list[i][2],change_list[i][4]]
        for j in range(2):
            triangles.append([j,diagonal[0], diagonal[1], change_list[i][1+j*2]])
    return triangles

In [12]:
def adjust_rectangles(rectangles,l,boolean):
    #takes in a collection of rectangles 
    #returns the same list but without 3 bottom right squares if boolean==0
    #return the same list but without  bottom right square if boolean==1
    if boolean ==0 :
        rectangles=np.delete(rectangles, 2*(l-1)-1, 0)
        rectangles=np.delete(rectangles, l-2, 0)
        rectangles=np.delete(rectangles, l-3, 0)
    if boolean ==1 :
        rectangles=np.delete(rectangles, l-2, 0)
    rectangles=pushback_index(rectangles)
    return rectangles

In [13]:
def adjust_edges (l, edges, triangles):
    #takes in the list of egdes and the collection of triangles
    #adds the diagonals of the triangles and remove bottom right corner edges
    for i in range(2): #we'll add the new edges first
        edges.append([triangles[i*2][1],triangles[i*2][2]])
    edges=find_and_remove_edges (l,edges)#then we remove
    edges=pushback_index(edges)
    return edges

In [14]:
def adjust_boundary(l, neumann, dirichlet, p_r):
    #takes in the previous boundary arrays and the previous rectangle collection
    #removes bottom right corner from neumann, adds corner to dirichlet
    neumann=find_and_remove_edges (l,neumann)
    for i in range(2):
        dirichlet.append([p_r[l-2][1+(i+2)%4],p_r[l-2][1+(i+3)%4]])
    return [pushback_index(neumann), pushback_index(dirichlet)]

In [15]:
def angle_mesh(dimension, angle):
    l=math.ceil(dimension)+1
    mesh=square_mesh(dimension) #we do everything like a square mesh
    #but we will make adjustments to the bottom right corner
    grid=adjust_grid(l, angle, mesh[0])
    triangles=generate_triangles (mesh[2], l)
    rectangles=adjust_rectangles(mesh[2],l,0)
    edges=adjust_edges (l, mesh[3], triangles)
    boundary=adjust_boundary(l, mesh[4], mesh[5], mesh[2]) #[0]neumann, [1]dirichlet
    ##
    return [grid, pushback_index(triangles), rectangles, pushback_index(edges), boundary[0], boundary[1]]


So far I've needed 6 functions to generate an mesh with a re-entrant corner of non-right angle. I'll need a special function for the 90 degrees one since there are also no triangles in that mesh. EDIT : One thing I forgot to take care of was the re-indexing. It's nice if I'm not skipping a number in my indexing so I have to re-index all the rectangles past the first one. 

In [16]:
def corner_mesh(dimension):
    mesh=square_mesh(dimension) #we do everything like a square mesh
    l=math.ceil(dimension)+1
    grid=add_index((np.delete(mesh[0], l-1,0)).tolist())#we delete the bottom right node
    rectangles=adjust_rectangles(mesh[2],l,1)
    edges=find_and_remove_edges (l,mesh[3])
    boundary=adjust_boundary(l, mesh[4], mesh[5], mesh[2])
    return [grid, mesh[1], rectangles, pushback_index(edges.tolist()), boundary[0], boundary[1]]

In [17]:
def generate_mesh (dimension, corner, angle):
    #input : length, boolean, angle in degrees
    #returns an array [coordinates[], triangles[], rectangles[], edges[], neumann[], dirichlet[]]
    mesh=[]
    if corner ==1:
        if abs(angle-90.)<19. :# the math.tan() function blows up if too close 
                  # to a right angle so we will treat those as 90deg angles
                mesh= corner_mesh(dimension)
        else:
            mesh=angle_mesh(dimension, angle)
    else : 
        mesh=square_mesh(dimension)
    return mesh

In hindsight it may not be very useful to have a dimension input, since what we really want for a more precise solution to the PDE is a tighter grid. However it didnt cost me much to implement this extra degree of freedom and it enable me to further test the code. Also note that because dimension is assigned in my console cell, it is a global variable so I technically do not need to pass it on anywhere (I will come back and make the code prettier if I have time).

In [18]:
dimension=3.0 #float, length of the square that contains the mesh
corner=0 #acts as boolean
angle=45. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 it'll be interpreted as a right angle
#####
mesh=generate_mesh(dimension, corner, angle)

Now we want to implement a mesh refinement algorithm that will basically split every triangle into 4 smaller triangles and every rectangles in 4 smaller rectangles as well.
I already coded something like that a few weeks ago but the challenge here is to make sure that it works with the input format I have from my mesh generating function. Unsurprisingly I ran into a lot of troubles between np arrays and lists so I think I'll reformat my output to lists before doing anything. EDIT : since I had to go back into my mesh generating functino to account for the shift in indexes my outputs are back in list format. But that's only if I have a corner mesh. So I need an if statement to reformat the output in case of a square mesh.

Mesh Refinement :

In [19]:
def gcent (nodes):
    #takesin array of coordinates of an element
    #returns coordinates of center of gravity of element
    l = len(nodes) #either 2, 3 or 4
    x=0
    y=0
    for i in range(l): #either 0-1, 0-2 or 0-3
        x+= nodes[i][1]
        y+=nodes[i][2]
    x=x/l
    y=y/l
    return [x,y]

The next 2 functions are really long but I haven't really found a way around it and since nothing is very re-usable there isn't really a point in splitting them any further.It's really just a bit of tidious and carefull array manipulations unfortunately.

In [20]:
def split_rect (rectangle, coordinates): #this works well
    nodes = [coordinates[rectangle[1]-1],coordinates[rectangle[2]-1]
             ,coordinates[rectangle[3]-1],coordinates[rectangle[4]-1]]
    midpoints=[]
    new=[]
    for i in range(4):
        mid= gcent([nodes[(i)%4], nodes[(i+1)%4] ]) 
        temp = [len(coordinates)+1, mid[0], mid[1]]
        already=0
        for j in range(len(coordinates)): #check this midpoint isnt already a node
            test=coordinates[j]
            if test[1]==temp[1]:
                if test[2]==temp[2]: #if it is already a node
                    already=1
                    temp=test #will use it to build new rectangles
        if already ==0:
            coordinates.append(temp)   
        midpoints.append(temp[0])
    coordinates.append([len(coordinates)+1,gcent(nodes)[0],gcent(nodes)[1]])
    midpoints.append(len(coordinates))
    for i in range(4):
        new_rect = [ i, nodes[i][0], midpoints[i], midpoints[4], midpoints[(i-1)%4]  ]
        new.append(new_rect)
    return [new,coordinates]

In [21]:
def split_tri(triangle, coordinates): #this is GOOD
    nodes=[coordinates[triangle[1]-1],coordinates[triangle[2]-1],coordinates[triangle[3]-1]]
    midpoints=[]
    new=[]
    for i in range(3):
        mid = gcent([nodes[(i)%3],nodes[(i+1)%3]])  
        temp=[len(coordinates)+1, mid[0], mid[1]]
        already=0
        for j in range(len(coordinates)): #check this midpoint isnt already a node
            test=coordinates[j]
            if test[1]==temp[1]:
                if test[2]==temp[2]: #if it is already a node
                    already=1
                    temp=test #will use it to build new triangles   
        if already ==0:
            coordinates.append(temp)
        midpoints.append(temp[0])
    for i in range(4):
        if i==3:
            new_tri=[i, midpoints[0],midpoints[1],midpoints[2]]
        else:
            new_tri=[i, nodes[i][0], midpoints[i], midpoints[(i-1)%3]]     
        new.append(new_tri)
    return [new,coordinates]

In [22]:
def split_shapes (triangles, rectangles, coordinates):
    #takes in the shapes lists and the list of coordinates
    #returns the lists of refined rectangles and triangles as well as an updated
    #list of coordinates 
    new_tri=[]
    new_rect=[]
    for i in range(len(triangles)):
        couple=split_tri(triangles[i], coordinates) # [0] new triangles, [1] updated coord.
        for j in range(len(couple[0])):
            new_tri.append(couple[0][j])
        coordinates=couple[1]
    for i in range(len(rectangles)):
        couple=split_rect(rectangles[i], coordinates)
        for j in range(len(couple[0])):
            new_rect.append(couple[0][j])
        coordinates=couple[1]
    new_tri=add_index(new_tri)
    new_rect=add_index(new_rect)
    return [new_tri,new_rect, coordinates]

Note : So far I've successfully fixed both splitting functions. Now I need to find a way to make the boundary refinement function work. Originally I was considering a fixed mesh so I was using the geometry of the corner to evaluate wether or not a point was on the dirichlet boundary or not. So is no longer possible since I'm customising the angle of that corner. However since we retain some amount of symetry I can possibly adjust the previous functions. Another approach would be to write a new function that evaluates wether or not the edge is part of an egde in the previous boundary. That is more straightforward and I'm wondering if the implementation has any tricks to it.

In [23]:
def refine_boundary (boundary, edges, coordinates):
    #takes in a list of refined edges and a collection of bondary edges
    # collects the refined edges that are part of the boundary
    new=[]
    for i in range(len(edges)):
        test=edges[i]
        for j in range(len(boundary)):
            g=gcent([coordinates[boundary[j][0]-1],coordinates[boundary[j][1]-1]])
            temp_1=boundary[j][0]
            temp_2=boundary[j][1]
            if test[1]==temp_1:
                if [coordinates[test[0]-1][1],coordinates[test[0]-1][2]]==g:
                    new.append(test)
            if test[1]==temp_2:
                if [coordinates[test[0]-1][1],coordinates[test[0]-1][2]]==g:
                    new.append(test)
            if test[0]==temp_1:
                if [coordinates[test[1]-1][1],coordinates[test[1]-1][2]]==g:
                    new.append(test)
            if test[0]==temp_2:
                if [coordinates[test[1]-1][1],coordinates[test[1]-1][2]]==g:
                    new.append(test)
    return new

In [24]:
def mesh_refinement (coordinates, triangles, rectangles, prev_boundary): #Im fairly convinced this works
    new_shapes=split_shapes (triangles, rectangles, coordinates)
    edges=generate_edges(new_shapes[0]+new_shapes[1]) #this is very good. 
    new_n=refine_boundary(prev_boundary[0], edges, new_shapes[2])
    new_d=refine_boundary(prev_boundary[1], edges, new_shapes[2])
    return [new_shapes[2], new_shapes[0], new_shapes[1], edges, new_n, new_d]

In [25]:
dimension=3.0 #float, length of the square that contains the mesh
corner=0 #acts as boolean
angle=45. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 it'll be interpreted as a right angle
#####
mesh=generate_mesh(dimension, corner, angle)
mesh=mesh_refinement(mesh[0], mesh[1], mesh[2],[mesh[4],mesh[5]])

UPDATE : It finally works. I'm contempt.

I'm debating wether I should create a new notebook to fix the PDE solver now or if I should just keep going on here. I don't know if I'll be able to call functions from another notebook once I push them on GitHub, that is my main concern. I guess I'll just have two notebooks for now and I can always manage this later. EDIT : Nevermind it's not working out I'm going to keep writting in here.

PDE Solver :

In [26]:
def f(x):
    #Volume Force
    #takes in a point (x, y)
    #returns f(x,y) in R
    f=(1/32)*(3*x[0]*x[1]*(4-x[1])-x[0]*x[0]*x[0])
    #f=1
    if Q5 == 1 :
        f=1
    return f

In [27]:
def g(x):
    #Neumann Boundary Condition
    #takes in a point (x,y)
    #returns g(x,y) in R
    g=-7.
    print ("I shouldn't be here")
    if x[1]==0.0: #along x-axis
        g=0.
    else :
        if x[0] == 4.0:# along x=4.0
            g=2*(2-x[1])
        else :
                print ("error in Neumann g")
    return g

In [28]:
def h(x):
    #Dirichlet Boundary Condition
    #takes in a point (x,y)
    #returns h(x,y) in R
    h=-7.
    if x[0]==0.0 : #along y axis
        h=0.
    else :
        if x[1]==dimension: #along y=dimension
            h=0.
        else :
            #print ("error in Dirichlet h")
            if x[1]==0.0: #along x-axis
                h=0.
            else :
                if x[0] == dimension:# along x=dimension
                    h=x[1]*(4-x[1])
                else :
                    if corner ==1 :
                        if x[0]==3.:
                            if x[1] < 1.05:
                                h=27/64*x[1]*(4-x[1])
                            else:
                                print ("error in Dirichlet h")
                        else:
                            if x[1]==1.:
                                if x[0]>2.99:
                                    h=3/64*x[0]*x[0]*x[0]
                                else :
                                    print ("error in Dirichlet h")
                            else:        
                                print ("error in Dirichlet h")
                    else:
                        print ("error in Dirichlet h")
            
    return h

The 3 previous functions are problem-based and can be modified easily to solve any combinations of problems.

Functions To Assemble Stiffness Matrix

In [29]:
def orient (shape, grid): #I was getting an extra minus sign without this
    #takes in a shape array
    #returns the correctly oriented array
    k=1
    somme=grid[shape[1]-1][1]+grid[shape[1]-1][2]
    for i in range(len(shape)-2):
        temp=shape[i+2]
        t_sum=grid[temp-1][1]+grid[temp-1][2]
        if t_sum < somme : #finds the node in the shape
            somme=t_sum     # closest to (0,0)
            k=i+2
    new=[shape[0]]
    for i in range(len(shape)-1):
        new.append(shape[1+(k+i-1)%(len(shape)-1)])
    return new

In [30]:
def area(shape, grid):
    #takes in a list of nodes and the list of associated coordinates
    #returns (the area of the shape) actually returns the matrix
    #this way I can re-use the function
    matrix=[[grid[shape[2]-1][1]-grid[shape[1]-1][1],grid[shape[len(shape)-1]-1][1]-grid[shape[1]-1][1]],
            [grid[shape[2]-1][2]-grid[shape[1]-1][2],grid[shape[len(shape)-1]-1][2]-grid[shape[1]-1][2]]]
    return matrix            


In [31]:
def compute_G (triangle, grid):
    #takes in a list of nodes for a triangle, and the list of coordinates
    #returns a matrix G= [[1,1,1], [x_1,x_2,x_3], [y_1, y_2, y_3]] . [[0,0],[1,0],[0,1]]
    G=[[1,1,1],
       [grid[triangle[1]-1][1], grid[triangle[2]-1][1], grid[triangle[3]-1][1]],
       [grid[triangle[1]-1][2], grid[triangle[2]-1][2], grid[triangle[3]-1][2]]]
    G=np.dot(np.linalg.inv(G), [[0,0],[1,0],[0,1]] )
    return G
    

In [32]:
def add_to_stiffness(A, M, shape):
    #takes in A, a smaller square matrix M and an indexed list of nodes
    #returns A with entries at (nodes[i],nodes[j]) incremented by values in <
    for i in range(len(shape)-1):
        for j in range(len(shape)-1):
            A[shape[i+1]-1][shape[j+1]-1]+=M[i][j]
    return A

In [33]:
def stiff3 (triangle, grid):
    #takes in the nodes of a triangle and the list of coordinates of nodes
    #returns a 3x3 stiffness matrix for that triangle
    matrix=area(orient(triangle, grid),grid)
    G=compute_G(triangle,grid)
    return abs(np.linalg.det(matrix))/2*(np.dot(G,np.transpose(G)))

In [34]:
def stiff4 (rectangle, grid) :
    #takes in the nodes of a rectangle and the list of coordinates of nodes
    #returns a 3x3 stiffness matrix for that rectangle
    matrix=area(orient(rectangle, grid), grid)
    B=np.linalg.inv(np.transpose(matrix)*matrix)
    C_1= B[0][0]*np.array([[2,-2],[-2,2]])+B[0][1]*np.array([[3,0],[0,-3]])+B[1][1]*np.array([[2,1],[1,2]])
    C_2= B[0][0]*np.array([[-1,1],[1,-1]])+B[0][1]*np.array([[-3,0],[0,3]])+B[1][1]*np.array([[-1,-2],[-2,-1]])
    M= abs(np.linalg.det(matrix))/6* np.block([ [C_1,C_2], [C_2, C_1] ])
    return M


In [35]:
def generate_stiffness (grid, triangles, rectangles):
    #takes in grid, triangles, rectangles
    #returns a len(grid)xlen(grid) stiffness matrix
    A=[[0. for i in range(len(grid))] for j in range(len(grid))]#initialising
    for i in range(len(triangles)):
        M=stiff3(triangles[i], grid)
        A=add_to_stiffness(A,M,triangles[i])
    for i in range(len(rectangles)):
        M=stiff4(rectangles[i], grid)
        A=add_to_stiffness(A,M,rectangles[i])
    return A

So far I've needed 6 functions to generate the left hand side of the PDE, I'm pretty happy with how the stiffness matrix part of the code turned out. It's definitely an upgrade from my previous version, in my opinion. EDIT : 7 functions, I noticed that not orienting my square properly was generating a bug further down. Fixed it !

Functions that compute the right hand side vector b

In [36]:
def add_to_rhs (b, value, shape):
    #takes in b, a value to be added, and the indexes of which b_j to increment
    #returns updated b
    for i in range(len(shape)-1):
        b[shape[i+1]-1]+=value
    return b

In [37]:
def VolumeForces (b,shapes, grid):
    #takes in the combined list of rectangles and triangles + coordinates
    #returns the updated right hand side
    for i in range(len(shapes)):
        T=abs(np.linalg.det(area(shapes[i], grid)))
        nodes=[]
        for j in range(len(shapes[i])-1):
            nodes.append(grid[shapes[i][j+1]-1])
        f_s=f(gcent(nodes))
        chi=(4-len(shapes[i])+1)*6+(len(shapes[i])-1-3)*4
        b=add_to_rhs(b,(1/chi)*T*f_s, shapes[i])
    return b

In [38]:
def length (edge, grid):
    #takes in an edge (node 1, node 2) and the list of coordinates
    #returns the length of the edge
    a=[grid[edge[0]-1][1], grid[edge[0]-1][2]]
    b=[grid[edge[1]-1][1], grid[edge[1]-1][2]]
    l=math.sqrt( (a[0]-b[0])*(a[0]-b[0]) +(a[1]-b[1])*(a[1]-b[1]) )
    return l

In [39]:
def NeumannBC (b, neumann, grid):
    #takes in the list of edges on Neumann Bdry
    #returns updated rhs vector
    for i in range(len(neumann)):
        g_m=g(gcent([grid[neumann[i][0]-1], grid[neumann[i][1]-1]]))
        l=length(neumann[i], grid)
        b=add_to_rhs(b, (l/2)*g_m, [i]+neumann[i])
    return b

In [40]:
def generate_rhs (grid, shapes, neumann):
    #takes in the combined list of rectangles and triangles plus the neumann bdry
    #returns a 1x len(grid) vector for stress+volume forces
    b= [0. for i in range(len(grid))]
    b=VolumeForces (b, shapes, grid)
    b=NeumannBC (b, neumann, grid)
    return b

I also needed 6 functions for the right hand side. Now I have to tackle the tricky part where I need to re-arrange A and b so that the nodes are split between free first and boundary nodes last. I think that's where I ran into troubles last time. I first tried to see if I could directly compute A and b with a reorganised grid but I would need to re-index all my shapes first and it just seem like a lot of work. Should I try to implement that in the mesh generation part of the code instead? That's a thought. But a lot of my code relies on the way my coordinates are indexed so I think that would create a lot of unwanted effects.

In [41]:
def compute_Ud (grid, notfixed):
    #takes in the list of coordinates and the list of dirichlet nodes
    #returns the values at those nodes
    Ud=[]
    for i in range(len(notfixed)):
        value=h([grid[notfixed[i]-1][1], grid[notfixed[i]-1][2]])
        Ud.append(value)
    return Ud

I think now what I really need is to re-index all my mesh. I'll try and see if it's doable and if that solves my problems. EDIT : The following 5 functions take care of that. It wasn't hard to implement but I'm still not sure that it'll fix my problems.

Functions That Re-Order the Mesh

In [42]:
def find (test, ogrid):
    #takes in an integer to find in ogrid
    #returns the index of that integers in ogrid
    k=-1
    for i in range(len(ogrid)):
        temp=ogrid[i][0]
        if temp ==test:
            k=i
    return k

In [43]:
def update_list (liste, ogrid):
    #takes in a list from the original mesh and the organised grid
    #returns the updated mesh element
    new_liste=[]
    for i in range(len (liste)):
        element=liste[i]
        if len(element)==2:
            new=[]
        else :
            new=[element[0]]
        for j in range(len(element)-len(new)):
            k=find (element[j+len(new)-j], ogrid)
            new.append(k+1)
        new_liste.append(new)
    return new_liste

In [44]:
def generate_ogrid (grid, dirichlet, ordered):
    ogrid=[]
    for i in range (len(ordered)):
        ogrid.append(grid[ordered[i]-1])
    return ogrid

In [45]:
def generate_free(boundary, grid):
    #takes in a list of boundary edges and a list of nodes
    #returns two lists, one with the free nodes, one with the bdry nodes
    free=[]
    notfree=[]
    for i in range(len(grid)):
        test=grid[i][0]#extract node
        append=1
        for j in range(len(boundary)):
            for k in range(2):
                temp=boundary[j][k]
                if temp==test:#if that node shows up in boundary
                    append=0
        if append ==1:
            free.append(test)
        else :
            notfree.append(test)
    return [free, notfree]

In [46]:
def update_mesh (mesh, organised):
    ogrid=generate_ogrid (mesh[0], mesh[5], organised[0]+organised[1] )
    omesh=[ogrid]
    for i in range (len(mesh)-1):
        omesh.append(update_list (mesh[i+1], ogrid))
    return omesh

The previous manipulation seemed to work so now I just need to extract the block matrices and solve for the values at the free nodes! Quite excited to see if it works.

Functions that Solve the PDE

In [47]:
def extract (A,b,Ud):
    #takes in the stiffness matrix, the RHS and the values at dirichlet nodes
    #returns relevant block matrices
    A11=[[]for i in range(len(A)-len(Ud))]
    for i in range(len(A)-len(Ud)):
        for j in range(len(A)-len(Ud)):
            A11[i].append(A[i][j])
    A12= [[]for i in range(len(A)-len(Ud))]
    for i in range(len(A)-len(Ud)):
        for j in range(len(Ud)):
            A12[i].append(A[i][len(A)-len(Ud)+j])
    bfree=[]
    for i in range(len(A)-len(Ud)):
        bfree.append(b[i])
    return [A11, A12, bfree]

In [48]:
def Free_Solve (A, b, Ud):
    #takes in the stiffness matrix, the RHS and the values at dirichlet nodes
    #returns value of the PDE on free nodes
    arr = extract (A,b,Ud)
    U=np.dot(np.linalg.inv(arr[0]), arr[2]-np.dot(arr[1], Ud))
    return U

In [49]:
def PDE_Solve (mesh):
    #takes in [coordinates, triangles, rectangles, edges, neumann, dirichlet]
    #returns values at free nodes given \nabla(u)= f and g on boundary
    organised=generate_free(mesh[5], mesh[0])
    omesh=update_mesh (mesh, organised)
    A=generate_stiffness(omesh[0] ,omesh[1] ,omesh[2])
    b= generate_rhs (omesh[0], omesh[1]+omesh[2], omesh[4])
    Ud=compute_Ud (mesh[0], organised[1])
    U=Free_Solve (A, b, Ud)                
    return [organised[0], U]

In [50]:
dimension=4.0 #float, length of the square that contains the mesh
corner=0 #acts as boolean
angle=110. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 it'll be interpreted as a right angle
#####
mesh=generate_mesh(dimension, corner, angle)
solution=PDE_Solve(mesh)
#print (mesh[0]) #grid
#print ("---")
#print (mesh[1]) #triangles
#print ("---")
#print (mesh[2]) #rectangles
#print ("---")
#print (mesh[3]) #edges
#print ("---")
#print (mesh[4]) #neumann
#print ("---")
#print (mesh[5]) #dirichlet
#print ("---")
#print ("free nodes :",solution[0])
#print ("---")
#print ("Values :", solution [1])
#print ("---")

I shouldn't be here
I shouldn't be here
I shouldn't be here
I shouldn't be here
I shouldn't be here
I shouldn't be here
I shouldn't be here
I shouldn't be here


So this is still bugged, I'm not getting a refinement of the values when I refine the mesh. I don't know what's wrong. I'm quite frustrated. Actually I don't think it is "very" wrong but it's just hard to see since I haven't invested time into making the solution look nice. I just don't understand how the values are getting bigger. It's like I'm adding more things and they are not getting smaller with the refinement. 

EDIT : So I've just written the orient() function and it's actually giving me hope. We note that now the values from the first few free nodes are still close to one another after mesh refinement. Could it be possible that I have successfully coded this ?

-------------------------------------------

Question 1 : Pick a convex domain \Omega and a smooth function u, compute f=-\nabla u in \Omega. Then solve the Poisson problem with the computed f and with g=u on d\Omega, on several grid with decreasing mesh sizes.

I chose u(x,y)= (x^{3}/32 )*y(4-y). I computed f and g and implemented them in the previous cells, see the beginning of the PDE Solver block. I'm solving on a 4x4 square mesh.

I wrote a couple functions that analyse the output so that I have an idea of what going on.

In [51]:
def u (x):
    #takes in a point (x,y)
    #returns the value of u(x,y) in R
    u=x[1]*(4-x[1])*x[0]*x[0]*x[0]*(1/64)
    return u

In [52]:
def compute_actual (free, grid):
    #takes in the list of free nodes and coordinates
    #returns the actual value of the solution at these nodes
    actual=[]
    for i in range(len(free)):
        value=u([grid[free[i]-1][1],grid[free[i]-1][2]])
        actual.append(value)
    return actual

In [53]:
def compute_inf_err (experimental, actual):
    #takes in 2 arrays of solution
    #computes the point by point difference
    error=[]
    for i in range(len(actual)):
        error.append(abs(experimental[i]-actual[i]))
    return error

In [54]:
def error_average (error):
    #takes in a list of point by point error estimates
    #return the average value
    average=0.0
    for i in range(len(error)):
        average+=error[i]
    return average/len(error)

In [55]:
def order_u_values (nodes, values):
    #takes in a shuffled list of nodes andsimilarly shuffled list of values
    #returns the ordered list of values
    u_array=[0.0 for i in range(len(nodes))]
    for i in range(len(u_array)):
        k=nodes[i]
        u_array[k-1]=values[i]
    return u_array

The particulat example that I chose works best without neumann boundary conditions. Thus I made slight changes to my PDE Solver to make that work.

In [56]:
def PDE_Solve_no_neumann (mesh):
    #takes in [coordinates, triangles, rectangles, edges, neumann, dirichlet]
    #returns values at free nodes given \nabla(u)= f and g on boundary
    organised=generate_free(mesh[5]+mesh[4], mesh[0])
    omesh=update_mesh (mesh, organised)
    A=generate_stiffness(omesh[0] ,omesh[1] ,omesh[2])
    b=generate_rhs(omesh[0], omesh[1]+omesh[2], [])
    Ud=compute_Ud (mesh[0], organised[1])
    U=Free_Solve (A, b, Ud)                
    return [organised[0], U, Ud, organised[1]]

In [57]:
dimension=4.0 #float, length of the square that contains the mesh
corner=0 #acts as boolean
angle=90. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 it'll be interpreted as a right angle
#####
repeat =3 # we will refine twice
#####
mesh=generate_mesh(dimension, corner, angle)
for i in range(repeat):
    solution=PDE_Solve_no_neumann(mesh)
    #
    actual=compute_actual (solution[0], mesh[0])
    error=compute_inf_err (solution [1], actual)
    av=error_average (error)
    print ("---")
    print ("on average :", av)
    #u_array=order_u_values (solution[0]+solution[3], solution[1].tolist()+solution[2])
    #print ("---")
    #print ("list of point by point solution :", u_array)
    #
    mesh=mesh_refinement(mesh[0], mesh[1], mesh[2],[mesh[4],mesh[5]])
print ("---")

---
on average : 0.29270833333333307
---
on average : 0.27703854335861644
---
on average : 0.25256566005298325
---


Question 2 : Plot the H^{1}-error of the finite element solution against the meshsize h on a logarthmic scale, and estimate the convergence rate \alpha in error ~ h^{\alpha} from the graph. Is it consistent with theory ?

To compute the H1 error I first need to compute linear interpolations between each nodes (:=p), then take a derivative (:=p') , and finally take the L2 norm of |u"-p'|. Since I'll have one interpolant per edge the integral is actually going to be a sum of integrals, for each edge.

In [58]:
def du (x):
    #takes in a point (x,y)
    #returns du(x,y), du the derivative of u du =u_x+u_y
    value=3/64*x[1]*(4-x[1])*x[0]*x[0]+x[0]*x[0]*x[0]*(1/64)*(4-2*x[1])
    return value

In [59]:
def orient_edge (edge, grid):
    #takes in an edge array
    #returns the correctly oriented array
    k=0
    somme=grid[edge[0]-1][1]+grid[edge[0]-1][2]
    temp=grid[edge[1]-1][1]+grid[edge[1]-1][2]
    if temp < somme : #finds the node in the shape
        somme=temp     # closest to (0,0)
        k=1
    new=[]
    for i in range(len(edge)):
        new.append(edge[(k+i)%2])
    return new

Ultimately I want the next function to be adaptable for non square mesh. I'm storing the result as [m, l, a] where p = mx +ly +a. This is where I'm having trouble I think. Somehow along the right side of the rectangle the derivative of the solution doesn't match the actual value. Is there a bug in my PDE Solver? I couldn't find anything so I think I'm just not processing my solution correctly.

In [60]:
def square_mesh_interp (edge, grid, u_array):
    #takes in an edge that is vertical or horizontal
    #returns the line equation in the format [m, l, a]
    k=1 #by default assume horizontal
    if grid[edge[0]-1][1]-grid[edge[1]-1][1] ==0: #check if vertical
        k=2
    array=[0.,0.,0.]
    array[k-1]=(u_array[edge[1]-1]-u_array[edge[0]-1])/(grid[edge[1]-1][k]-grid[edge[0]-1][k])
    array[2]=u_array[edge[1]-1]-(array[k-1]*grid[edge[1]-1][k])
    return array
    

In [61]:
def generate_interp (edges, grid, u_array):
    #takes in the list of all edges in the mesh
    #returns the list of coeff of linear interpolants for each edge
    p_list=[]
    for i in range(len(edges)):
        temp=[i+1]+square_mesh_interp (edges[i], grid, u_array)
        p_list.append(temp)
    return p_list

In [62]:
def generate_dp (p_list):
    #takes in the list of arrays of the form [k, m, l, a]
    #returns [k, m, l] the tangents
    dp_list=[]
    for i in range(len(p_list)):
        dp_list.append([p_list[i][0], p_list[i][1], p_list[i][2]])
    return dp_list

In [63]:
def dp (a, x):
    #takes in coefficient a[] and a point x[] in R^{2}
    #returns value of dp(x)
    value=a[0]+a[1] #for a square mesh one of those is going to be zero
    return value

This next one might also have problems, it's very tidiously written because I wanted to be sure I was handling the right data.

In [64]:
def compute_H1norm (grid, edges, dp_list) :
    #takes in the list of tangeant coefficients
    #returns the values of int(|du-dp|^{2}) on each edge
    norm_list=[]
    for i in range(len(edges)):#print ("This is the edge :", edges[i])
        a=[grid[edges[i][0]-1][1], grid[edges[i][0]-1][2]]
        b=[grid[edges[i][1]-1][1], grid[edges[i][1]-1][2]]#print ("a is :", a)#print ("b is :", b)#mx=(a[0]+b[0])/2#my=(a[1]+b[1])/2
        coeff=[dp_list[i][1], dp_list[i][2]]#print ("the coefficients are ", coeff)
        norm=0.
        if b[0]-a[0]==0.:#print ("vertical edge")#print ("dp is :", dp(coeff, [a[0],my]) )#print ("du is :", du([a[0],my]) )
            norm=quad(lambda y: (abs(dp(coeff, [a[0],y])-du([a[0],y]))*abs(dp(coeff, [a[0],y])-du([a[0],y]))),
                              a[1], b[1])#print ("norm :", norm)
        else :#print ("horizontal edge")#print ("dp is :", dp(coeff, [a[0],mx]) )#print ("du is :", du([a[0],mx]) )
            norm=quad(lambda x: (abs(dp(coeff, [x,a[1]])-du([x,a[1]]))*abs(dp(coeff, [x,a[1]])-du([x,a[1]]))),
                              a[0], b[0])#print ("norm :", norm)
        norm_list.append(norm) 
    somme=0.0
    for i in range(len(norm_list)):
        somme+=norm_list[i][0]
    return somme

In [65]:
def H1_norm (grid, edges, solution):
    u_array=order_u_values (solution[0]+solution[3], solution[1].tolist()+solution[2])
    oriented_edges=[]
    for i in range(len(edges)):
        edge=orient_edge(edges[i], grid)
        oriented_edges.append(edge)
    p_list=generate_interp (oriented_edges, grid, u_array)
    dp_list=generate_dp (p_list)
    norm = compute_H1norm (grid, edges, dp_list)
    return norm

In [66]:
dimension=4.0 #float, length of the square that contains the mesh
corner=0 #acts as boolean
angle=110. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109
              #it'll be interpreted as a right angle
#####
repeat =3 # we will refine twice
#####
mesh=generate_mesh(dimension, corner, angle)
for i in range(repeat):
    solution=PDE_Solve_no_neumann(mesh)
    H1=H1_norm(mesh[0], mesh[3], solution)
    print ("H1 Norm :",math.sqrt(H1)) 
    print ("Mesh Size :", length(mesh[3][i], mesh[0]))
    mesh=mesh_refinement(mesh[0], mesh[1], mesh[2],[mesh[4],mesh[5]])
    

H1 Norm : 4.788138442965687
Mesh Size : 1.0
H1 Norm : 3.138607622189701
Mesh Size : 0.5
H1 Norm : 2.7448078174603023
Mesh Size : 0.25


This is obviously not correct. I did try to find where the bug was and noticed I had consistent overshoot along the right side of the mesh, where the function takes non zero values. I think what's happening is that I'm not interpolating correctly. I'm quite sad that this is working, but I don't really have time to go back and fix it unfortunately. At least it's decreasing with the mesh size.

Question 3 : Repeat the preceding exercise for the L^{2}-error of the finite element solution. Explain the results.

In [67]:
def p(a, x):
    #takes in a list of coefficient a[] and point x[]
    #returns value of interpolant
    value=a[0]*x[0]+a[1]*x[1]+a[2]#print (a)#print (x)
    return value

In [68]:
def compute_L2norm (grid, edges, p_list) :
    norm_list=[]
    for i in range(len(edges)):#print ("This is the edge :", edges[i])
        a=[grid[edges[i][0]-1][1], grid[edges[i][0]-1][2]]
        b=[grid[edges[i][1]-1][1], grid[edges[i][1]-1][2]]#print ("a is :", a)#print ("b is :", b)#mx=(a[0]+b[0])/2#my=(a[1]+b[1])/2
        coeff=[p_list[i][1], p_list[i][2], p_list[i][3]]#print ("the coefficients are ", coeff)
        norm=0.
        if b[0]-a[0]==0.:#print ("vertical edge")#print ("dp is :", dp(coeff, [a[0],my]) )#print ("du is :", du([a[0],my]) )
            norm=quad(lambda y: (abs(p(coeff, [a[0],y])-du([a[0],y]))*abs(p(coeff, [a[0],y])-du([a[0],y]))),
                            a[1], b[1])#print ("norm :", norm)
        else :#print ("horizontal edge")#print ("p is :", p(coeff, [mx, a[1]]) )#print ("u is :", u([mx, a[1]]))
            norm=quad(lambda x: (abs(p(coeff, [x,a[1]])-u([x,a[1]]))*abs(p(coeff, [x,a[1]])-u([x,a[1]]))),
                            a[0], b[0])#print ("norm :", norm)
        norm_list.append(norm) 
    somme=0.0
    for i in range(len(norm_list)):
        somme+=norm_list[i][0]
    return somme

In [69]:
def L2_norm (grid, edges, solution):
    u_array=order_u_values (solution[0]+solution[3], solution[1].tolist()+solution[2])
    oriented_edges=[]
    for i in range(len(edges)):
        edge=orient_edge(edges[i], grid)
        oriented_edges.append(edge)
    p_list=generate_interp (oriented_edges, grid, u_array)
    norm=compute_L2norm (grid, edges, p_list)
    return norm

In [70]:
dimension=4.0 #float, length of the square that contains the mesh
corner=0 #acts as boolean
angle=110. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 
             #it'll be interpreted as a right angle
#####
repeat =3 
#####
mesh=generate_mesh(dimension, corner, angle)
for i in range(repeat):
    solution=PDE_Solve_no_neumann(mesh)
    L2=L2_norm (mesh[0], mesh[3], solution)
    print ("L2 Norm :", math.sqrt(L2))
    print ("Mesh Size :", length(mesh[3][i], mesh[0]))
    mesh=mesh_refinement(mesh[0], mesh[1], mesh[2],[mesh[4],mesh[5]])

L2 Norm : 5.093696265107953
Mesh Size : 1.0
L2 Norm : 5.035934827499937
Mesh Size : 0.5
L2 Norm : 4.452702257237994
Mesh Size : 0.25


Unsurprisingly this is also wrong, but we note that it's much bigger in magnitude which makes sense since we are in a Sobolev space so the H1 norm should be more precise than the L2 norm.

Question 4: Repeat the 3 preceding exercises for a smooth function u on a domain with a re-entrant corner. Explain the results.

In [71]:
dimension=4.0 #float, length of the square that contains the mesh
corner=1 #acts as boolean
angle=90. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 it'll be interpreted as a right angle
#####
repeat =3 
#####
mesh=generate_mesh(dimension, corner, angle)
for i in range(repeat):
    solution=PDE_Solve_no_neumann(mesh)
    #print ("grid :", mesh[0])
    #print ("rectangles :",mesh[2])
    #print ("edges :", mesh[3])
    #print ("dirichlet :", mesh[4])
    #
    actual=compute_actual (solution[0], mesh[0])
    error=compute_inf_err (solution [1], actual)
    av=error_average (error)
    print ("---")
    print ("on average :", av)
    H1=H1_norm(mesh[0], mesh[3], solution)
    print ("H1 Norm :",math.sqrt(H1)) 
    print ("Mesh Size :", length(mesh[3][i], mesh[0]))
    L2=L2_norm (mesh[0], mesh[3], solution)
    print ("L2 Norm :", math.sqrt(L2))
    print ("Mesh Size :", length(mesh[3][i], mesh[0]))
    mesh=mesh_refinement(mesh[0], mesh[1], mesh[2],[mesh[4],mesh[5]])
print ("---")

---
on average : 0.32422249880038384
H1 Norm : 3.6644359038172625
Mesh Size : 1.0
L2 Norm : 4.219043750983692
Mesh Size : 1.0
---
on average : 0.28824338942572114
H1 Norm : 3.010760393947853
Mesh Size : 0.5
L2 Norm : 4.050904855412736
Mesh Size : 0.5
---
on average : 0.2600749781406311
H1 Norm : 2.7119388126000166
Mesh Size : 0.25
L2 Norm : 3.671860043364236
Mesh Size : 0.25
---


I'm a bit worried because I was expecting bigger error with a corner mesh. The domain is less smooth so the approximation should be less good. I wonder if this is just a consequence of the overshoot for a square mesh or if there' another error somewhere else. We note that the average point by point error is bigger here than on the square mesh example.

Question 5 : In a domain with a re-entrant corner, consider a solution of the form u(r, \theta)=r^{a}sin(a*\theta), where the coordinate origin is at the re-entrant corner, andd the rays \theta =0, \theta= \pi/a correspond to the two sides of the corner. For example, for an L-shaped domain, we would have a=2/3. Repeat exercises 1-3. 

So I have already written a code for the angle mesh. However the coordinates do not really match since my re-entrant corner is at the bottom right corner. This being said I think I can get away with write a function that translates all the coordinates so that (0,0) is at the bottom right node. I might get in trouble with negative coordinates though because I recall having written some functions just assuming to get only positive values in there. We shall see.

In [77]:
def translate_grid (grid):
    #takes in the grid
    #returns it so the the origin is at the re-entrant corner
    l=math.ceil(dimension)+1
    point=mesh[0][2*(l-1)-1]
    grid=[]
    for i in range(len(mesh[0])):
        a=mesh[0][i][1]-point[1]
        b=mesh[0][i][2]-point[2]
        grid.append([i+1, a, b])
    return grid

The previous function does the translating. Now I have to figure out where it is safe to insert it. I think right before doing the PDE solving is good, because in the mesh generator I think I need to have the x=0,y=0, x=dim, y=dim criteria to define the boundary. So I'm going to write a slightly different PDE Solver for this particular problem.

In [79]:
def PDE_Solve_Q5_no_neumann (mesh):
    #takes in [coordinates, triangles, rectangles, edges, neumann, dirichlet]
    #returns values at free nodes given \nabla(u)= f and g on boundary
    organised=generate_free(mesh[5]+mesh[4], mesh[0])
    omesh=update_mesh (mesh, organised)
    omesh[0]=translate_grid (omesh[0])
    A=generate_stiffness(omesh[0] ,omesh[1] ,omesh[2])
    b=generate_rhs(omesh[0], omesh[1]+omesh[2], [])
    Ud=compute_Ud (mesh[0], organised[1])
    U=Free_Solve (A, b, Ud)                
    return [organised[0], U, Ud, organised[1]]

I also have to take care of the new f,g,h so I'll declare a global variable Q5 so that I can directly get a new if statement in the f,g,h cells, instead of writting completely new functions.

In [78]:
dimension=4.0 #float, length of the square that contains the mesh
corner=1 #acts as boolean
angle=45. #float, angle of the corner in degrees between 0 and 180.
            # note that if angle is between 71 and 109 
              #it'll be interpreted as a right angle
Q5=1 #acts as boolean
#####
repeat =3 # we will refine twice
#####
mesh=generate_mesh(dimension, corner, angle)

##translate everything
grid=translate_grid (mesh[0])
print (grid)
#
##hope it goes well

[[1, -3.0, -1.0], [2, -2.0, -1.0], [3, -1.0, -1.0], [4, 0.557851739352194, -1.0], [5, -3.0, 0.0], [6, -2.0, 0.0], [7, -1.0, 0.0], [8, 0.0, 0.0], [9, 1.0, -0.5578517393521941], [10, -3.0, 1.0], [11, -2.0, 1.0], [12, -1.0, 1.0], [13, 0.0, 1.0], [14, 1.0, 1.0], [15, -3.0, 2.0], [16, -2.0, 2.0], [17, -1.0, 2.0], [18, 0.0, 2.0], [19, 1.0, 2.0], [20, -3.0, 3.0], [21, -2.0, 3.0], [22, -1.0, 3.0], [23, 0.0, 3.0], [24, 1.0, 3.0]]


I need to switch to cartesian coordinates for the f,g,h functions and it's more work than I thought so I'm actually not going to be able to finish today. Very soon though.